In [4]:
import torch
import torch.nn as nn
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os
from PIL import Image, ImageDraw
import numpy as np
import torchvision.transforms as T


#DATAPATH = "data/val/"


transform = T.Compose([
    T.ToTensor(),
])


class LaysDetector:
    def __init__(self, imgPath):
        self.model = self.get_model(num_classes=2)
        self.imgPath = imgPath
        self.boxes = []
        self.prediction = None

    def get_model(self, num_classes):
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

        model.load_state_dict(torch.load("models/detection_model.zip" , map_location=torch.device('cpu')))                

        return model


    def get_prediction(self, img):
        with torch.no_grad():
            self.prediction = self.model([img])

        return self.prediction        



    def draw_detection(self):
        img = self.imgPath
        ans_image = []
        self.model.eval()

        img = Image.open(img).convert("RGB")
        img = transform(img)
 #       print(type(img)) 
#        print(img.shape)

        prediction = self.get_prediction(img)



        image = Image.fromarray(img.mul(255).permute(1,2,0).byte().numpy())
        draw = ImageDraw.Draw(image)

#        print(prediction)

        for element in range(len(prediction[0]["boxes"])):
            boxes = prediction[0]["boxes"][element].cpu().numpy()
#            print(boxes)
            score = np.round(prediction[0]["scores"][element].cpu().numpy(), decimals= 4)
   
            if score > 0.6:
                draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
                outline ="green", width =10)
                draw.text((boxes[0], boxes[1]), text = str(score))
            
        image.show()

    def get_boxes(self):
        img = self.imgPath
        ans_image = []
        self.model.eval()

        img = Image.open(img).convert("RGB")
        img = transform(img)
#        print(type(img)) 
#        print(img.shape)

        prediction = self.get_prediction(img)
        #print(prediction)
        return prediction[0]["boxes"].cpu().numpy()


        



if __name__ == "__main__":
    detector = LaysDetector(imgPath= "test.jpeg")
    boxes = detector.get_boxes()
    print(boxes)
    

    detector.draw_detection()

C:\Users\gurutva.patle\.conda\envs\tensorflow env\lib\site-packages\torchvision\ops\boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


[[ 50.150085    9.998729  149.70949   152.98384  ]
 [133.63129     7.3066473 249.98709   158.74596  ]]
